In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
cd /content/gdrive/My\ Drive/vip2

/content/gdrive/My Drive/vip2


In [0]:
train_dir = 'NWPU-RESISC12/train'
validation_dir = 'NWPU-RESISC12/test'
image_size = 224
nTrain = 6600
nVal = 1800

In [7]:
import numpy as np
import pickle
from keras.models import load_model

vgg_places = load_model('vgg_places_30_epochs.h5')
print(vgg_places.summary())

Using TensorFlow backend.


OSError: ignored

In [24]:
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, load_img
import keras
from keras import optimizers


# Specify the data augmentation techniques in the trainig image data generator
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 10
val_batchsize = 10

# Data Generator for Training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')


Found 6600 images belonging to 12 classes.


In [25]:
validation_datagen = ImageDataGenerator(rescale=1./255)

# Data Generator for Validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


Found 1800 images belonging to 12 classes.


In [0]:

# Compile the model
vgg_places.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])



In [0]:
# Train the Model
# NOTE that we have multiplied the steps_per_epoch by 2. This is because we are using data augmentation.
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = vgg_places.fit_generator(
      train_generator,
      steps_per_epoch=2*train_generator.samples/train_generator.batch_size ,
      epochs=40,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      callbacks=[es_callback],
      verbose=1
      )



Epoch 1/40
1320/1320 [==============================] - 2561s 2s/step - loss: 0.7263 - acc: 0.8470 - val_loss: 0.6993 - val_acc: 0.8711
Epoch 2/40
1320/1320 [==============================] - 275s 209ms/step - loss: 0.7556 - acc: 0.8426 - val_loss: 1.4510 - val_acc: 0.7717
Epoch 3/40
1320/1320 [==============================] - 287s 217ms/step - loss: 0.7999 - acc: 0.8355 - val_loss: 0.6196 - val_acc: 0.8761
Epoch 4/40
1320/1320 [==============================] - 300s 227ms/step - loss: 0.8640 - acc: 0.8211 - val_loss: 0.9318 - val_acc: 0.8022
Epoch 5/40
1320/1320 [==============================] - 293s 222ms/step - loss: 0.8829 - acc: 0.8255 - val_loss: 0.9950 - val_acc: 0.8211
Epoch 6/40
1320/1320 [==============================] - 278s 211ms/step - loss: 0.9220 - acc: 0.8126 - val_loss: 0.8785 - val_acc: 0.8372
Epoch 7/40
1320/1320 [==============================] - 286s 216ms/step - loss: 0.9649 - acc: 0.8092 - val_loss: 0.8799 - val_acc: 0.8589
Epoch 8/40
1320/1320 [==============

In [0]:
# Save the Model
vgg_places.save('vgg_places_data_aug.h5')


In [0]:
vgg_places.evaluate(validation_generator)

180/180 [==============================] - 23s 130ms/step


[0.9102938946750428, 0.8549999949004915]

pwd

In [17]:
ls gdrive/My\ Drive/Colab\ Notebooks/vip2


 BOW.ipynb                           VGG_Bottleneck_Train_Labels.pkl
 bow_vgg.ipynb                       VGG-FineTune-DA.h5
 codebook.pkl                        VGG-FineTune-DataAugementation
 descriptors_test.npy                VGG-FineTune.h5
 descriptors_train.npy               VGG-FineTuning
 Lab12.ipynb                         vgg-fullyconnected.h5
 NWPU-RESISC12/                      VGG_FullyConnected.ipynb
 NWPUvip.zip                         vgg_places_30_epochs.h5
 __pycache__/                        vgg_places_and_16.ipynb
 res-classifier.h5                   VGG_Places_Bottleneck_Test_Features.pkl
 RES_FineTune.ipynb                  VGG_Places_Bottleneck_Test_Labels.pkl
 ResNetFineTune.h5                   VGG_Places_Bottleneck_Train_Features.pkl
 ResNetFineTune_weights.h5           VGG_Places_Bottleneck_Train_Labels.pkl
 restnet.ipynb                       vgg_places_data_aug.h5
 vgg16_places_365.py                 vgg_places_data_aug.ipynb
 vgg16_places_history.png     

In [18]:
cd gdrive/My\ Drive/Colab\ Notebooks/vip2

/content/gdrive/My Drive/Colab Notebooks/vip2


In [19]:
model = load_model('vgg_places_data_aug.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

def scores(y_true,y_pred):
  accuracy = accuracy_score(y_true,y_pred)
  precision = precision_score(y_true,y_pred,average='macro')
  recall = recall_score(y_true,y_pred,average='macro')
  f1 = f1_score(y_true,y_pred,average='macro')
  confusionmatrix = confusion_matrix(y_true, y_pred)
  print("Accuracy:  {}".format(accuracy))
  print("Precision: {}".format(precision))
  print("Recall:    {}".format(recall))
  print("F1:        {}".format(f1))
  print("Confusion matrix:\n{}".format(confusionmatrix))

In [0]:
predictions = model.predict_generator(validation_generator)
predicted_classes = np.argmax(predictions,axis=1)
#test_labels = np.argmax(test_labels,axis=1)

In [0]:
y_true = validation_generator.classes

In [33]:
scores(y_true,predicted_classes)

Accuracy:  0.855
Precision: 0.8624825543754112
Recall:    0.855
F1:        0.8554058945579427
Confusion matrix:
[[113   0   6   1   1   0  12   5   5   0   2   5]
 [  0 136   0   0   0   8   0   0   2   2   0   2]
 [  0   0 141   0   1   0   0   0   6   0   0   2]
 [  0   0   1 130   0   0   1   1   6   3   6   2]
 [  0   0   0   0 145   0   0   1   1   0   2   1]
 [  0   7   0   1   0 141   0   0   1   0   0   0]
 [  1   0   0   0   0   0 136   2   0   0   6   5]
 [  1   0   0   0   0   0   0 128   3   0   4  14]
 [  0   0   6   0   4   0   0   0 132   0   6   2]
 [  0   3   4   0   6   4   0   0   1 116   5  11]
 [  0   1   1   5   2   0   1  14  16   0 104   6]
 [  1   0   0   0  10   0   0  11   5   0   6 117]]
